In [5]:
# Set Variables
SW_LAT = -38.88917
SW_LON = 140.84444
NE_LAT = -33.70833
NE_LON = 150.0
# Getting monthly mean!
PARAMETER_NAME = "Pat4_C_B_1_MonthlyMean"
PROPERTY_NAME = "Water Course Discharge"
TIME_PERIOD = "2010/2024"
OUTPUT_DOC = "MonthlyMeanWaterCourseDischarge"


In [6]:
import requests
from xml.etree.ElementTree import Element, SubElement, tostring

# Define the namespace map to use in the XML
ns = {
    'sos': "http://www.opengis.net/sos/2.0",
    'fes': "http://www.opengis.net/fes/2.0",
    'gml': "http://www.opengis.net/gml/3.2",
    'xsi': "http://www.w3.org/2001/XMLSchema-instance",
    'sams': "http://www.opengis.net/samplingSpatial/2.0"
}

# Define the endpoint URL for the SOS service
url = "http://www.bom.gov.au/waterdata/services"

# Create XML elements for the request
envelope = Element('sos:GetFeatureOfInterest', {
    'service': 'SOS', 'version': '2.0', 'xmlns:sos': ns['sos'], 'xmlns:fes': ns['fes'], 'xmlns:gml': ns['gml'], 'xmlns:xsi': ns['xsi'], 'xmlns:sams': ns['sams']
})

spatial_filter = SubElement(envelope, 'sos:spatialFilter')
bbox = SubElement(spatial_filter, 'fes:BBOX')
value_ref = SubElement(bbox, 'fes:ValueReference')
value_ref.text = 'om:featureOfInterest/*/sams:shape'

envelope_ = SubElement(bbox, 'gml:Envelope', {
    'srsName': 'urn:ogc:def:crs:EPSG::4326'
})
lower_corner = SubElement(envelope_, 'gml:lowerCorner')
upper_corner = SubElement(envelope_, 'gml:upperCorner')

# Insert the actual coordinates for the Murray River bounding box
lower_corner.text = f"{SW_LAT} {SW_LON}"  # Southernmost, westernmost
upper_corner.text = f"{NE_LAT} {NE_LON}"  # Northernmost, easternmost

# Add observed property element
observed_property = SubElement(envelope, 'sos:observedProperty')
observed_property.text = f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}"  # Example observed property


# Convert the ElementTree to a string
request_xml = tostring(envelope, 'utf-8')

# Define headers for the POST request
headers = {
    'Content-Type': 'application/xml'
}

# Send the POST request
FeatureOfInterestResponse = requests.post(url, data=request_xml, headers=headers)

# Print the response from the server


In [7]:
import requests
import xml.etree.ElementTree as ET
# Sample XML response from GetFeatureOfInterest
xml_response = FeatureOfInterestResponse.text

# Parse the XML string
root = ET.fromstring(xml_response)

# Extract station IDs
station_ids = [elem.text.split('/')[-1] for elem in root.findall('.//gml:identifier', namespaces={'gml': 'http://www.opengis.net/gml/3.2'})]

print(station_ids) 

#get stations within region


['403213', '403239', '403251', '412060', '412094', '403249', '410178', '410177', '600162', '405327', '407285', '407289', '41200209', '412028', '412053', '412051', '412066', '225213', '225213A', '225100A', '405209', '407238', '407238A', '410061', '410132', '410042', '410198', '410038', '227211', '233250', '235209', '235219', '227216', '229650A', '229650B', '235256', '41000271', '425054', '229250A', '235222', '235278', '235260', '217006', '405721', '405753', '235205', '229107', '229104C', '229628A', '229620A', '229272A', '229251A', 'A2390546', '408202', '408206', '408212', '408203', '408209', '408219', '408200', '408213', '225224', '225224A', '225201A', '225201', '225234', '225234B', '415220', '225255A', '225255', '230235A', '406214', '406262', '406216', '405336', 'WP220', 'WP221', 'WP222', 'WP223', 'WP224', 'WP225', '410171', '410091', '41010810', '410186', '410170', '231706A', '231716A', '412169', '412072', '407257', '402212', '401520', 'BAGOG1', 'BAGOG3', 'BAGOG4', 'BAGOG5', '232709',

In [8]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

# Define the SOS2 endpoint URL
url = "http://www.bom.gov.au/waterdata/services"

data = []  # List to store data for DataFrame


# Define the parameters for the GetObservation request
params = {
    'service': 'SOS',
    'version': '2.0.0',
    'request': 'GetObservation',
    'procedure': f"http://bom.gov.au/waterdata/services/tstypes/{PARAMETER_NAME}",
    'observedProperty': f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}",
    'temporalFilter': f"om:phenomenonTime,{TIME_PERIOD}"
}

# Assuming station_ids is defined earlier in the script
for station_id in station_ids:
    print(f"Querying data for station: {station_id}")
    
    # Add the station ID to the parameters
    params['featureOfInterest'] = f"http://bom.gov.au/waterdata/services/stations/{station_id}"
    
    # Make the GetObservation request to the SOS2 endpoint
    response = requests.get(url, params=params, timeout=100)  # Added timeout
    
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)
        
        # Find all the MeasurementTVP elements which contain the time-value pairs
        tvp_elements = root.findall('.//{http://www.opengis.net/waterml/2.0}MeasurementTVP')
        
        if not tvp_elements:
            print("No data found for this station.")
        else:
            # Print out the time and value for each MeasurementTVP
            for tvp in tvp_elements:
                time = tvp.find('{http://www.opengis.net/waterml/2.0}time').text
                value = tvp.find('{http://www.opengis.net/waterml/2.0}value').text
                #print(f"  {time}: {value}")
                data.append({'Station': station_id, 'Time': time, 'Value': value})
    else:
        print(f"Failed to retrieve data for station {station_id}. HTTP status code: {response.status_code}")
        print("Response Headers:", response.headers)
        print("Response Body:", response.text)


df = pd.DataFrame(data)


Querying data for station: 403213
Querying data for station: 403239
No data found for this station.
Querying data for station: 403251
Querying data for station: 412060
No data found for this station.
Querying data for station: 412094
No data found for this station.
Querying data for station: 403249
Querying data for station: 410178
No data found for this station.
Querying data for station: 410177
No data found for this station.
Querying data for station: 600162
Querying data for station: 405327
Querying data for station: 407285
Querying data for station: 407289
Querying data for station: 41200209
Querying data for station: 412028
Querying data for station: 412053
No data found for this station.
Querying data for station: 412051
No data found for this station.
Querying data for station: 412066
Querying data for station: 225213
Querying data for station: 225213A
Querying data for station: 225100A
No data found for this station.
Querying data for station: 405209
Querying data for station:

In [9]:


conv_df = df.copy()
conv_df.set_index('Time', inplace=True)


pivot_df = df.pivot_table(index='Time', values='Value', columns='Station', aggfunc='first')


pivot_df.index = pd.to_datetime(pivot_df.index)



# Resample data into hourly bins and calculate the mean for each hour
monthly_df = pivot_df.resample('M').first()

# Show pivoted DataFrame
print(monthly_df)

monthly_df.to_csv(f'{OUTPUT_DOC}.csv', index=True)

Station                   212040 212060 2122518 2122711 2122713 2122725  \
Time                                                                      
2010-01-31 00:00:00+10:00  0.010   None    None   0.677    None    None   
2010-02-28 00:00:00+10:00  0.085   None    None   5.301    None    None   
2010-03-31 00:00:00+10:00  0.026   None    None   0.080    None    None   
2010-04-30 00:00:00+10:00  0.024   None    None   0.028    None    None   
2010-05-31 00:00:00+10:00  6.494   None    None   0.104    None    None   
...                          ...    ...     ...     ...     ...     ...   
2023-09-30 00:00:00+10:00  0.133  0.152   0.251   0.628    None    None   
2023-10-31 00:00:00+10:00  0.057  0.045   0.046   0.203    None    None   
2023-11-30 00:00:00+10:00  0.038  0.020    None   1.392    None    None   
2023-12-31 00:00:00+10:00  0.048  0.771    None   3.138    None    None   
2024-01-31 00:00:00+10:00  0.161  0.655   0.789   2.696    None    None   

Station                 